In [4]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests
import re

In [76]:
from nba_api.stats.static import teams as nba_teams_module
from nba_api.stats.endpoints.commonteamroster import CommonTeamRoster
from nba_api.stats.endpoints.leaguestandingsv3 import LeagueStandingsV3
from nba_api.stats.endpoints.playercompare import PlayerCompare
from nba_api.stats.endpoints import playercareerstats, playergamelog
from nba_api.stats.endpoints import winprobabilitypbp

In [74]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

# Create Team and Roster Class

In [149]:
class NBATeamRosters:
    def __init__(self, season):
        """
        Initializes the class and sets up the teams DataFrame and an empty dictionary for rosters.
        Args:
            season (str): The NBA season in 'YYYY' format, e.g., '2024' for the 2024-2025 season.
        """
        self.season = season
        self.teams_df = None
        self.rosters = {}
        self.standings_df = None
        self.schedule_df = None
        self.player_stats = {}
        self.fetch_teams()

    def fetch_teams(self):
        """
        Fetches NBA teams and stores them in a DataFrame.
        """
        nba_teams = nba_teams_module.get_teams()
        self.teams_df = pd.DataFrame(nba_teams).set_index('id')

    def fetch_rosters(self):
        """
        Fetches and stores the rosters for each team.
        The rosters are stored in a dictionary with keys as the team's abbreviation, nickname, and city.
        """
        for team_id, team_data in self.teams_df.iterrows():
            team_roster = self.get_team_roster(team_id)
            if team_roster is not None:
                self.rosters[team_data['abbreviation']] = team_roster
                self.rosters[team_data['nickname']] = team_roster
                self.rosters[team_data['city']] = team_roster

    def get_team_roster(self, team_id):
        """
        Fetches the roster for a single team based on the team ID.
        Args:
            team_id (int): The team's ID.

        Returns:
            pd.DataFrame: DataFrame containing the team's roster.
        """
        try:
            rosters_raw = CommonTeamRoster(
                team_id=str(team_id),
                season=self.season
            ).common_team_roster.get_data_frame()
            return rosters_raw
        except Exception as e:
            print(f"Error fetching roster for team ID {team_id}: {e}")
            return None

    def get_roster_by_key(self, key):
        """
        Retrieves the roster DataFrame by the team's abbreviation, nickname, or city.
        Args:
            key (str): The team's abbreviation, nickname, or city.

        Returns:
            pd.DataFrame: DataFrame containing the team's roster.
        """
        return self.rosters.get(key, None)

    def load_schedule(self, csv_path):
        """
        Loads the NBA schedule from a CSV file into the class.
        Args:
            csv_path (str): The file path to the schedule CSV file.
        """
        self.schedule_df = pd.read_csv(csv_path)
        print(f"Schedule loaded: {len(self.schedule_df)} games")

    def get_teams_for_game(self, game_id):
        """
        Retrieves the teams playing in a specific game from the schedule.
        Args:
            game_id (str): The ID of the game.
    
        Returns:
            Tuple[pd.DataFrame, pd.DataFrame]: DataFrames for the two teams playing the game.
        """
        # Ensure the 'Game ID' column is treated as a string
        self.schedule_df['Game ID'] = self.schedule_df['Game ID'].astype(str)
        
        # Check for the game ID in the schedule
        game_row = self.schedule_df.loc[self.schedule_df['Game ID'] == game_id]
        
        if game_row.empty:
            raise ValueError(f"Game ID {game_id} not found in the schedule.")
        
        game = game_row.iloc[0]
        home_team = self.teams_df[self.teams_df['abbreviation'] == game['Home Team Abbreviation']]
        away_team = self.teams_df[self.teams_df['abbreviation'] == game['Visiting Team Abbreviation']]
        
        return home_team, away_team

    def fetch_team_stats(self, team_abbreviation, season):
        """
        Fetches team statistics for a given team abbreviation and season.
        Args:
            team_abbreviation (str): The team abbreviation (e.g., 'GSW' for Golden State Warriors).
            season (str): The NBA season in 'YYYY-YY' format (e.g., '2023-24').
    
        Returns:
            dict: A dictionary containing the team's statistics.
        """
        # Map the team abbreviation to a team ID (this should be predefined or fetched)
        team_id = self.get_team_id_from_abbreviation(team_abbreviation)
        
        if not team_id:
            print(f"Team ID not found for abbreviation {team_abbreviation}.")
            return {}
    
        # Fetch team stats using the NBA API
        team_stats = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(
            team_id=team_id,
            season=season,
            season_type_all_star='Regular Season'
        )
        
        # Convert the stats into a dictionary or DataFrame
        team_stats_dict = team_stats.overall_team_dashboard.get_data_frame().iloc[0].to_dict()
        
        return team_stats_dict

    
    def get_todays_games(self, today):
        """
        Retrieves the games scheduled for a specific date.
        Args:
            today (str): The date in 'YYYY-MM-DD' format.

        Returns:
            pd.DataFrame: DataFrame containing the games scheduled for the specified date.
        """
        if self.schedule_df is not None:
            todays_games = self.schedule_df[self.schedule_df["Game Date"] == today]
            return todays_games
        else:
            print("Schedule not loaded.")
            return None

    def get_full_schedule(self):
        """
        Returns the full schedule DataFrame.
        Returns:
            pd.DataFrame: DataFrame containing the entire schedule.
        """
        if self.schedule_df is not None:
            return self.schedule_df
        else:
            print("Schedule not loaded.")
            return None
    
    def fetch_league_standings(self, season_type):
        """
        Fetches the league standings for the specified season and season type.
        Args:
            season_type (str): The type of season ('Regular Season', 'Playoffs', etc.)

        Returns:
            pd.DataFrame: DataFrame containing the league standings.
        """
        league_standings = LeagueStandingsV3(
            league_id="00",
            season=self.season,
            season_type=season_type
        )
        standings_data = league_standings.standings.get_dict()
        self.standings_df = pd.DataFrame(standings_data['data'], columns=standings_data['headers'])
        self.standings_df.index = np.arange(1, len(self.standings_df) + 1)
        self.standings_df = self.standings_df.drop(columns=['SeasonID', 'LeagueID', 'TeamSlug', 'LeagueRank'])
        self.standings_df = self.standings_df.rename(columns={'TeamID': 'id'})
        return self.standings_df

    def process_league_standings(self, season_type):
        """
        Processes the league standings by division and conference.
        Args:
            season_type (str): The type of season ('Regular Season', 'Playoffs', etc.)

        Returns:
            pd.DataFrame: DataFrame sorted by Conference and Division.
        """
        # Fetch and process the standings
        league_standings = self.fetch_league_standings(season_type)

        # Group by 'Division' and create separate DataFrames for each division
        grouped = league_standings.groupby('Division')
        divisions = {division_name: group for division_name, group in grouped}
        
        # Prepare and modify DataFrames for each division
        modified_dfs = []
        for division, df in divisions.items():
            # Create a separator row with the division name
            separator_df = pd.DataFrame([{col: '' for col in df.columns}])
            separator_df.iloc[0, df.columns.get_loc('Division')] = division
            
            # Concatenate the separator row and the division DataFrame
            combined_df = pd.concat([separator_df, df])
            
            # Append the combined DataFrame to the list
            modified_dfs.append(combined_df)
        
        # Combine all modified DataFrames
        final_df = pd.concat(modified_dfs, ignore_index=True)
        
        # Order by 'Conference' and set specific values
        conference_order = ['East', 'West']
        final_df['Conference'] = pd.Categorical(final_df['Conference'], categories=conference_order, ordered=True)
        
        # Set specific 'Conference' values based on the provided indices
        final_df.loc[0, 'Conference'] = "East"
        final_df.loc[6, 'Conference'] = "East"
        final_df.loc[12, 'Conference'] = "West"
        final_df.loc[18, 'Conference'] = "West"
        final_df.loc[24, 'Conference'] = "East"
        final_df.loc[30, 'Conference'] = "West"
        
        # Sort the DataFrame by 'Conference' and 'Division'
        final_df_sorted = final_df.sort_values(by=['Conference', 'Division'])
        
        return final_df_sorted

    def get_player_team(self, player_id):
        """
        Fetches the team abbreviation for a given player based on their player ID.
        Args:
            player_id (int): The ID of the player.
    
        Returns:
            str: The team abbreviation.
        """
        # Assuming `self.rosters` is a DataFrame containing the full roster information
        player_roster = None
        for team_roster in self.rosters.values():
            if not team_roster.empty:
                player_roster = team_roster[team_roster['PLAYER_ID'] == player_id]
                if not player_roster.empty:
                    break
    
        if player_roster is not None and not player_roster.empty:
            team_abbreviation = player_roster['TEAM_ABBREVIATION'].iloc[0]
            return team_abbreviation
        else:
            print(f"Team not found for player ID {player_id}.")
            return None
    
    def fetch_player_career_stats(self, player_id):
        """
        Fetches career statistics for a player.
        Args:
            player_id (int): The ID of the player.

        Returns:
            pd.DataFrame: DataFrame containing the player's career statistics.
        """
        try:
            career_stats = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]
            self.player_stats[player_id] = {'career_stats': career_stats}
            return career_stats
        except Exception as e:
            print(f"Error fetching career stats for player ID {player_id}: {e}")
            return None

    def fetch_player_game_logs(self, player_id, season):
        """
        Fetches game logs for a player for a specific season.
        Args:
            player_id (int): The ID of the player.
            season (str): The NBA season in 'YYYY-YY' format, e.g., '2024-25'.

        Returns:
            pd.DataFrame: DataFrame containing the player's game logs for the season.
        """
        try:
            game_logs = playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]
            if player_id not in self.player_stats:
                self.player_stats[player_id] = {}
            self.player_stats[player_id]['game_logs'] = game_logs
            return game_logs
        except Exception as e:
            print(f"Error fetching game logs for player ID {player_id} in season {season}: {e}")
            return None

    def get_player_stats(self, player_id):
        """
        Retrieves the stored stats for a specific player.
        Args:
            player_id (int): The ID of the player.

        Returns:
            dict: A dictionary containing the player's career stats and game logs.
        """
        return self.player_stats.get(player_id, None)

    def compare_players(self, home_player_id, visiting_player_id, season="2023-24"):
        """
        Compares two players using the NBA API's PlayerCompare endpoint.
        Args:
            home_player_id (int): The ID of the home player.
            visiting_player_id (int): The ID of the visiting player.
            season (str): The NBA season in 'YYYY-YY' format, e.g., '2023-24'.

        Returns:
            dict: A dictionary with individual comparison data and overall comparison data.
        """
        try:
            player_compare_api = PlayerCompare(
                vs_player_id_list=str(visiting_player_id),
                player_id_list=str(home_player_id),
                season=season
            )
            
            # Retrieve the data
            individual_data = player_compare_api.individual.get_data_frame()
            overall_compare_data = player_compare_api.overall_compare.get_data_frame()
            
            return {
                "individual_data": individual_data,
                "overall_compare_data": overall_compare_data
            }
        except Exception as e:
            print(f"Error comparing players {home_player_id} and {visiting_player_id}: {e}")
            return None

    def fetch_win_probability(self, game_id, run_type='each second'):
        """
        Fetches the win probability data for a specific game using the NBA API.
        Args:
            game_id (str): The ID of the game.
            run_type (str): The granularity of the data ('each second', 'each possession').

        Returns:
            dict: A dictionary with game information and win probability data.
        """
        try:
            win_prob_api = winprobabilitypbp.WinProbabilityPBP(
                game_id=game_id,
                run_type=run_type
            )
            
            game_info = win_prob_api.game_info.get_data_frame()
            win_prob_data = win_prob_api.win_prob_pbp.get_data_frame()
            
            return {
                "game_info": game_info,
                "win_probability": win_prob_data
            }
        except Exception as e:
            print(f"Error fetching win probability data for Game ID {game_id}: {e}")
            return None

    def get_win_probability(self, game_id, run_type='each second'):
        """
        Retrieves the win probability data for a specific game.
        Args:
            game_id (str): The ID of the game.
            run_type (str): The granularity of the data ('each second', 'each possession').

        Returns:
            pd.DataFrame: DataFrame containing the win probability data.
        """
        win_prob_data = self.fetch_win_probability(game_id, run_type)
        if win_prob_data:
            return win_prob_data['win_probability']
        else:
            return None

    def calculate_ts_percentage(self, player_stats):
        """
        Calculates True Shooting Percentage (TS%) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.

        Returns:
            float: The player's TS%.
        """
        pts = player_stats['PTS']
        fga = player_stats['FGA']
        fta = player_stats['FTA']
        return pts / (2 * (fga + 0.44 * fta))

    def calculate_efg_percentage(self, player_stats):
        """
        Calculates Effective Field Goal Percentage (eFG%) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.

        Returns:
            float: The player's eFG%.
        """
        fg = player_stats['FGM']
        fga = player_stats['FGA']
        three_pointers = player_stats['FG3M']
        return (fg + 0.5 * three_pointers) / fga

    def calculate_usg_percentage(self, player_id, season):
        """
        Calculates Usage Rate (USG%) for a player.
        Args:
            player_id (int): The ID of the player.
            season (str): The NBA season in 'YYYY-YY' format (e.g., '2023-24').
    
        Returns:
            float: The player's USG%.
        """
        # Step 1: Get player stats
        player_stats = self.get_player_stats(player_id)
        
        # Step 2: Identify the player's team
        team_abbreviation = self.get_player_team(player_id)
        
        if not team_abbreviation:
            print(f"Unable to find team for player ID {player_id}. Cannot calculate USG%.")
            return None
        
        # Step 3: Fetch team stats based on the team abbreviation
        team_stats = self.fetch_team_stats(team_abbreviation, season)
        
        # Step 4: Proceed with the USG% calculation
        fga = player_stats.get('FGA', 0)
        fta = player_stats.get('FTA', 0)
        tov = player_stats.get('TOV', 0)
        minutes = player_stats.get('MIN', 0)
    
        team_fga = team_stats.get('FGA', 0)
        team_fta = team_stats.get('FTA', 0)
        team_tov = team_stats.get('TOV', 0)
        team_minutes = team_stats.get('MIN', 0) * 5  # 5 players on the court
    
        if minutes == 0 or team_minutes == 0:
            return 0  # Return 0 if the minutes or team minutes are zero to avoid division by zero
    
        return 100 * ((fga + 0.44 * fta + tov) * team_minutes) / (minutes * (team_fga + 0.44 * team_fta + team_tov))



    def calculate_ppp(self, player_stats, possessions):
        """
        Calculates Points Per Possession (PPP) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            possessions (int): The number of possessions.
    
        Returns:
            float: The player's Points Per Possession (PPP).
        """
        pts = player_stats['PTS']
        return pts / possessions if possessions else 0

    def calculate_per(self, player_stats, team_stats, league_stats):
        """
        Calculates Player Efficiency Rating (PER) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            team_stats (pd.Series): A series containing the team's stats.
            league_stats (pd.Series): A series containing the league's stats.
    
        Returns:
            float: The player's Player Efficiency Rating (PER).
        """
        # Simplified formula focusing on main stats
        uPER = (
            player_stats['PTS']
            + player_stats['REB'] * 0.3
            + player_stats['AST'] * 0.3
            + player_stats['STL'] * 0.3
            + player_stats['BLK'] * 0.3
            - player_stats['TOV'] * 0.4
        )
        
        per = uPER * (team_stats['Pace'] / league_stats['Pace'])
        return per

    def calculate_pie(self, player_stats, team_stats, opponent_stats):
        """
        Calculates Player Impact Estimate (PIE) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            team_stats (pd.Series): A series containing the team's stats.
            opponent_stats (pd.Series): A series containing the opponent's stats.
    
        Returns:
            float: The player's Player Impact Estimate (PIE).
        """
        player_total = (
            player_stats['PTS'] + player_stats['FGM'] + player_stats['FTM']
            - player_stats['FGA'] - player_stats['FTA']
            + player_stats['OREB'] + player_stats['DREB']
            + player_stats['AST'] + player_stats['STL'] + player_stats['BLK']
            - player_stats['PF'] - player_stats['TOV']
        )
        
        team_total = (
            team_stats['PTS'] + team_stats['FGM'] + team_stats['FTM']
            - team_stats['FGA'] - team_stats['FTA']
            + team_stats['OREB'] + team_stats['DREB']
            + team_stats['AST'] + team_stats['STL'] + team_stats['BLK']
            - team_stats['PF'] - team_stats['TOV']
        )
        
        opponent_total = (
            opponent_stats['PTS'] + opponent_stats['FGM'] + opponent_stats['FTM']
            - opponent_stats['FGA'] - opponent_stats['FTA']
            + opponent_stats['OREB'] + opponent_stats['DREB']
            + opponent_stats['AST'] + opponent_stats['STL'] + opponent_stats['BLK']
            - opponent_stats['PF'] - opponent_stats['TOV']
        )
        
        pie = player_total / (team_total + opponent_total)
        return pie

    def calculate_ortg(self, player_stats, team_possessions):
        """
        Calculates Offensive Rating (ORtg) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            team_possessions (int): The total number of team possessions.
    
        Returns:
            float: The player's Offensive Rating (ORtg).
        """
        points_produced = player_stats['PTS'] + 0.5 * player_stats['AST']
        ortg = (points_produced / team_possessions) * 100
        return ortg

    def calculate_drtg(self, player_stats, opponent_possessions):
        """
        Calculates Defensive Rating (DRtg) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            opponent_possessions (int): The total number of opponent possessions.
    
        Returns:
            float: The player's Defensive Rating (DRtg).
        """
        points_allowed = player_stats['PTS_AGAINST']
        drtg = (points_allowed / opponent_possessions) * 100
        return drtg

    def calculate_orb_percentage(self, player_stats, team_stats, opponent_stats):
        """
        Calculates Offensive Rebound Percentage (ORB%) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            team_stats (pd.Series): A series containing the team's stats.
            opponent_stats (pd.Series): A series containing the opponent's stats.
    
        Returns:
            float: The player's ORB%.
        """
        orb = player_stats['OREB']
        team_orb = team_stats['OREB']
        opponent_drb = opponent_stats['DREB']
        orb_percentage = (orb * (team_stats['MIN'] * 5)) / (player_stats['MIN'] * (team_orb + opponent_drb))
        return orb_percentage
    
    def calculate_drb_percentage(self, player_stats, team_stats, opponent_stats):
        """
        Calculates Defensive Rebound Percentage (DRB%) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            team_stats (pd.Series): A series containing the team's stats.
            opponent_stats (pd.Series): A series containing the opponent's stats.
    
        Returns:
            float: The player's DRB%.
        """
        drb = player_stats['DREB']
        team_drb = team_stats['DREB']
        opponent_orb = opponent_stats['OREB']
        drb_percentage = (drb * (team_stats['MIN'] * 5)) / (player_stats['MIN'] * (team_drb + opponent_orb))
        return drb_percentage

    def calculate_ast_percentage(self, player_stats, team_stats):
        """
        Calculates Assist Percentage (AST%) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            team_stats (pd.Series): A series containing the team's stats.
    
        Returns:
            float: The player's AST%.
        """
        assists = player_stats['AST']
        team_field_goals = team_stats['FGM']
        ast_percentage = 100 * (assists / (team_field_goals - player_stats['FGM']))
        return ast_percentage
    
    def calculate_tov_percentage(self, player_stats):
        """
        Calculates Turnover Percentage (TOV%) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
    
        Returns:
            float: The player's TOV%.
        """
        tov = player_stats['TOV']
        fga = player_stats['FGA']
        fta = player_stats['FTA']
        tov_percentage = 100 * (tov / (fga + 0.44 * fta + tov))
        return tov_percentage

    def calculate_stl_percentage(self, player_stats, opponent_possessions):
        """
        Calculates Steal Percentage (STL%) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            opponent_possessions (int): The total number of opponent possessions.
    
        Returns:
            float: The player's STL%.
        """
        steals = player_stats['STL']
        stl_percentage = 100 * (steals * (team_stats['MIN'] * 5)) / (player_stats['MIN'] * opponent_possessions)
        return stl_percentage
    
    def calculate_blk_percentage(self, player_stats, opponent_possessions):
        """
        Calculates Block Percentage (BLK%) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            opponent_possessions (int): The total number of opponent possessions.
    
        Returns:
            float: The player's BLK%.
        """
        blocks = player_stats['BLK']
        blk_percentage = 100 * (blocks * (team_stats['MIN'] * 5)) / (player_stats['MIN'] * opponent_possessions)
        return blk_percentage

    def calculate_pace(self, team_stats, opponent_stats):
        """
        Calculates Team Pace.
        Args:
            team_stats (pd.Series): A series containing the team's stats.
            opponent_stats (pd.Series): A series containing the opponent's stats.
    
        Returns:
            float: The team's Pace.
        """
        team_possessions = team_stats['POSS']
        opponent_possessions = opponent_stats['POSS']
        minutes_played = team_stats['MIN']
        pace = 48 * ((team_possessions + opponent_possessions) / (2 * (minutes_played / 5)))
        return pace

    def calculate_offensive_efficiency(self, player_stats, team_possessions):
        """
        Calculates Offensive Efficiency.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            team_possessions (int): The number of team possessions.
    
        Returns:
            float: The team's Offensive Efficiency.
        """
        points = player_stats['PTS']
        return points / team_possessions if team_possessions else 0
    
    def calculate_defensive_efficiency(self, opponent_stats, opponent_possessions):
        """
        Calculates Defensive Efficiency.
        Args:
            opponent_stats (pd.Series): A series containing the opponent's stats.
            opponent_possessions (int): The number of opponent possessions.
    
        Returns:
            float: The team's Defensive Efficiency.
        """
        points_allowed = opponent_stats['PTS']
        return points_allowed / opponent_possessions if opponent_possessions else 0

    def calculate_bpm(self, player_stats, team_stats, league_stats):
        """
        Calculates Box Plus/Minus (BPM) for a player.
        Args:
            player_stats (pd.Series): A series containing the player's stats.
            team_stats (pd.Series): A series containing the team's stats.
            league_stats (pd.Series): A series containing the league's stats.
    
        Returns:
            float: The player's BPM.
        """
        # Simplified version focusing on points and assists
        bpm = (
            player_stats['PTS'] + player_stats['AST'] * 0.5
            - player_stats['TOV'] * 0.5 - player_stats['PF'] * 0.5
            + player_stats['STL'] * 0.5 + player_stats['BLK'] * 0.5
        ) / player_stats['MIN'] * 100
        
        # Adjust for team and league context
        bpm_adjusted = bpm * (team_stats['Pace'] / league_stats['Pace'])
        return bpm_adjusted

    def calculate_vorp(self, bpm, minutes_played):
        """
        Calculates Value Over Replacement Player (VORP).
        Args:
            bpm (float): The player's Box Plus/Minus.
            minutes_played (float): The number of minutes played by the player.
    
        Returns:
            float: The player's VORP.
        """
        replacement_level = -2.0  # Approximate replacement-level BPM
        vorp = (bpm - replacement_level) * (minutes_played / 2000)
        return vorp

    def calculate_dws(self, drtg, team_drtg, minutes_played, team_minutes):
        """
        Calculates Defensive Win Shares (DWS) for a player.
        Args:
            drtg (float): The player's Defensive Rating.
            team_drtg (float): The team's Defensive Rating.
            minutes_played (float): The player's minutes played.
            team_minutes (float): The team's total minutes played.
    
        Returns:
            float: The player's DWS.
        """
        marginal_defense = (team_drtg - drtg) * (minutes_played / team_minutes)
        dws = marginal_defense * (minutes_played / 48) * (team_minutes / 5)
        return dws


# Roster Search

In [82]:
# Example usage
nba_rosters = NBATeamRosters(season="2024")
nba_rosters.fetch_rosters()

In [ ]:
# Fetch and print the teams DataFrame
teams_df = nba_data.teams_df
teams_df

In [ ]:
# Retrieve roster by abbreviation, nickname, or city
lakers_roster = nba_rosters.get_roster_by_key('LAL')
lakers_roster

In [ ]:
celtics_roster = nba_rosters.get_roster_by_key('Celtics')
celtics_roster

In [ ]:
boston_roster = nba_rosters.get_roster_by_key('Boston')
boston_roster

# Standings

In [87]:
# Instantiate the class for the 2024-2025 season
nba_data = NBATeamRosters(season="2024")

In [ ]:
# Fetch and process league standings for the Regular Season
processed_standings = nba_data.process_league_standings(season_type="Regular Season")
processed_standings

# Schedule

In [96]:
# Load the schedule from a CSV file
nba_data.load_schedule("nbaSchedule24.csv")

Schedule loaded: 1200 games


In [97]:
todays_games = nba_data.get_todays_games('2024-10-22')
todays_games

,Game Date,Game ID,Game Time,Arena,Arena City,Home Team Abbreviation,Home Conference,Home Division,Visiting Team Abbreviation,Visiting Conference,Visiting Division,Divisional Game,Conference Game
0,2024-10-22,22400061,7:30 pm ET,TD Garden,Boston,BOS,East,Atlantic,NYK,East,Atlantic,Yes,Yes
1,2024-10-22,22400062,10:00 pm ET,Crypto.com Arena,Los Angeles,LAL,West,Pacific,MIN,West,Northwest,No,Yes


In [ ]:
# Get the full schedule
full_schedule = nba_data.get_full_schedule()
full_schedule

# Game Analyzer

In [99]:
home_team, away_team = nba_data.get_teams_for_game('22400061')

# Player Advanced Stats Data

In [150]:
# Example usage of advanced metrics
nba_data = NBATeamRosters(season="2024")

In [144]:
nba_data.fetch_player_career_stats(player_id=201939)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,201939,2009-10,00,1610612744,GSW,22.0,80,77,2896.0,528,1143,0.462,166,380,0.437,177,200,0.885,48,308,356,472,152,19,243,252,1399
1,201939,2010-11,00,1610612744,GSW,23.0,74,74,2489.0,505,1053,0.480,151,342,0.442,212,227,0.934,52,234,286,432,109,20,226,233,1373
2,201939,2011-12,00,1610612744,GSW,24.0,26,23,732.0,145,296,0.490,55,121,0.455,38,47,0.809,15,73,88,138,39,8,65,62,383
3,201939,2012-13,00,1610612744,GSW,25.0,78,78,2983.0,626,1388,0.451,272,600,0.453,262,291,0.900,59,255,314,539,126,12,240,198,1786
4,201939,2013-14,00,1610612744,GSW,26.0,78,78,2846.0,652,1383,0.471,261,615,0.424,308,348,0.885,46,288,334,666,128,14,294,194,1873
5,201939,2014-15,00,1610612744,GSW,27.0,80,80,2613.0,653,1341,0.487,286,646,0.443,308,337,0.914,56,285,341,619,163,16,249,158,1900
6,201939,2015-16,00,1610612744,GSW,28.0,79,79,2700.0,805,1598,0.504,402,886,0.454,363,400,0.908,68,362,430,527,169,15,262,161,2375
7,201939,2016-17,00,1610612744,GSW,29.0,79,79,2639.0,675,1443,0.468,324,789,0.411,325,362,0.898,61,292,353,524,142,17,239,183,1999
8,201939,2017-18,00,1610612744,GSW,30.0,51,51,1631.0,428,864,0.495,212,501,0.423,278,302,0.921,36,225,261,310,80,8,153,114,1346
9,201939,2018-19,00,1610612744,GSW,31.0,69,69,2331.0,632,1340,0.472,354,810,0.437,263,287,0.916,45,324,369,361,92,25,192,166,1881


In [145]:
player_data = nba_data.get_player_stats(201939)
if player_data:
    print(player_data)  # Print to see the structure and contents of the player's data
else:
    print("No data found for the player.")


{'career_stats':     PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  \
0      201939   2009-10        00  1610612744               GSW        22.0   
1      201939   2010-11        00  1610612744               GSW        23.0   
2      201939   2011-12        00  1610612744               GSW        24.0   
3      201939   2012-13        00  1610612744               GSW        25.0   
4      201939   2013-14        00  1610612744               GSW        26.0   
5      201939   2014-15        00  1610612744               GSW        27.0   
6      201939   2015-16        00  1610612744               GSW        28.0   
7      201939   2016-17        00  1610612744               GSW        29.0   
8      201939   2017-18        00  1610612744               GSW        30.0   
9      201939   2018-19        00  1610612744               GSW        31.0   
10     201939   2019-20        00  1610612744               GSW        32.0   
11     201939   2020-21        00  

# Player Stats

In [94]:
# Instantiate the class for the 2024-2025 season
nba_data = NBATeamRosters(season="2024")

# Fetch career stats for a specific player (replace 'player_id' with an actual ID)
career_stats = nba_data.fetch_player_career_stats(player_id=201939)  # Example: Stephen Curry's ID
print("Career Stats:")
print(career_stats)

# Fetch game logs for the 2024-25 season
game_logs = nba_data.fetch_player_game_logs(player_id=201939, season="2024-25")
print("Game Logs for 2024-25:")
print(game_logs)

# Retrieve all stored stats for a player
player_stats = nba_data.get_player_stats(player_id=201939)
print("Stored Player Stats:")
print(player_stats)

Career Stats:
    PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  \
0      201939   2009-10        00  1610612744               GSW        22.0   
1      201939   2010-11        00  1610612744               GSW        23.0   
2      201939   2011-12        00  1610612744               GSW        24.0   
3      201939   2012-13        00  1610612744               GSW        25.0   
4      201939   2013-14        00  1610612744               GSW        26.0   
5      201939   2014-15        00  1610612744               GSW        27.0   
6      201939   2015-16        00  1610612744               GSW        28.0   
7      201939   2016-17        00  1610612744               GSW        29.0   
8      201939   2017-18        00  1610612744               GSW        30.0   
9      201939   2018-19        00  1610612744               GSW        31.0   
10     201939   2019-20        00  1610612744               GSW        32.0   
11     201939   2020-21        00  161

In [92]:


# Suppose you have the player IDs from your rosters
home_player_id = 203957  # Example: Zach LaVine
visiting_player_id = 1628991  # Example: Jayson Tatum

# Compare the two players
comparison_results = nba_data.compare_players(home_player_id, visiting_player_id, season="2023-24")

# Access individual and overall comparison data
individual_comparison = comparison_results["individual_data"]
overall_comparison = comparison_results["overall_compare_data"]

print("Individual Comparison Data:")
print(individual_comparison)

print("Overall Comparison Data:")
print(overall_comparison)

Individual Comparison Data:
       GROUP_SET     DESCRIPTION          MIN  FGM   FGA  FG_PCT  FG3M  FG3A  \
0     INDIVIDUAL         D. Exum  1087.621667  161   302   0.533    53   108   
1  VS INDIVIDUAL  J. Jackson Jr.  2124.180000  516  1161   0.444   117   366   

   FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  AST  TOV  STL  BLK  BLKA   PF  \
0    0.491   53   68   0.779    25   124  149  157   49   22    5    18   85   
1    0.320  337  417   0.808    85   280  365  154  160   80  106   121  235   

   PFD   PTS  PLUS_MINUS  
0   78   428       239.0  
1  378  1486      -340.0  
Overall Comparison Data:
    GROUP_SET                      DESCRIPTION          MIN  FGM   FGA  \
0  VS OVERALL  J. Jackson Jr.'s combined stats  2124.180000  516  1161   
1     OVERALL         D. Exum's combined stats  1087.621667  161   302   

   FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  AST  TOV  \
0   0.444   117   366    0.320  337  417   0.808    85   280  365  154  160   
1  